# Prepare data for alignment

This will read in the metadata and match IMC panoramas with slidescan scenes.

Further it will export the slidescan scenes as individual images.



In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
Cin = snakemake.input
Cout = snakemake.output
Cparams = snakemake.params

In [ ]:
import pathlib
import pandas as pd
import numpy as np

In [ ]:
import workflow.scripts.utils_alignment.library as lib

import imctools.librarybase as imclib
import imctools.library as imclib2

In [ ]:
import skimage.transform as sktranf

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

## Config

In [ ]:
import workflow.scripts.utils_alignment.variables as V

In [ ]:
for f in [pathlib.Path(Cout.fol_out_imgs)]:
    f.mkdir(parents=True, exist_ok=True)

In [ ]:
dat_slidemeta = pd.read_csv(Cin.fn_slidematch)

In [ ]:
fns = [f.stem for f in pathlib.Path(Cin.fol_imc).glob('*') if f.is_dir()]

In [ ]:
dat_imcac = lib.get_imcfn_meta(fns, Cparams.re_imcac)
dat_imcac = dat_imcac.merge(dat_slidemeta[[V.SAMPLEBLOCK, V.SLIDENUMBER, V.SLIDE]],how='left')

In [ ]:
dat = list()
for fn_slide, dat_slide in dat_imcac.groupby(V.SLIDESCAN):
    dat_scene_cord = lib.get_scene_meta(pathlib.Path(Cin.fol_slide) / f'{fn_slide}.czi')
    dat.append(dat_scene_cord)
dat_scene_cord = pd.concat(dat).reset_index()

In [ ]:
d_pano = []
d_ac = []
d_acroi = []
for fn in dat_imcac[V.SESSIONID]:
    m = lib.ImcMeta(pathlib.Path(Cin.fol_imc) / fn)
    dat_pano_cord, dat_ac_cord = lib.get_imc_meta(m)
    d_pano.append(dat_pano_cord)
    d_ac.append(dat_ac_cord)
dat_pano_cord = pd.concat(d_pano).reset_index()
dat_ac_cord = pd.concat(d_ac).reset_index()

In [ ]:
lib.add_center(dat_pano_cord)

In [ ]:
params = np.loadtxt(Cin.fn_transf_ifslide_imcslide)
transf_ifslide_imcslide = sktranf.AffineTransform(params)

In [ ]:
lib.add_center(dat_scene_cord, transform=transf_ifslide_imcslide)

In [ ]:
lib.add_order(dat_scene_cord, V.SLIDE)

In [ ]:
dat_pano_cord = dat_pano_cord.apply(lib.get_match, axis=1, dat=dat_scene_cord, idcol=V.SCENEID)

In [ ]:
dat_pano_cord

In [ ]:
d = []

for grp, dat in dat_pano_cord.merge(dat_imcac[[V.SESSIONID, V.SLIDE]]).groupby(V.SLIDE):
    d.append(dat.apply(lib.get_match, axis=1,
                                 dat=dat_scene_cord.query(f'{V.SLIDE}=="{grp}"'), idcol=V.SCENEID))
dat_pano_cord = pd.concat(d)

In [ ]:
dat_pano_cord

In [ ]:
dat_pano_cord[V.SCENEID] = dat_pano_cord[V.SCENEID].astype(int)
dat_pano_cord[V.PANOID] = dat_pano_cord[V.PANOID].astype(int)

In [ ]:
dat_ac_cord

In [ ]:
dat_cords = (dat_ac_cord
             .merge(dat_pano_cord[[V.SESSIONID, V.PANOID, V.SCENEID, V.SLIDESCAN]], 
                    on=[V.SESSIONID, V.PANOID], suffixes=('','_pano'))
                 .merge(dat_scene_cord[[V.SLIDE, V.SCENEID]])
            )

In [ ]:
dat_scene_cord.to_csv(Cout.fn_scenecords, index=False)

In [ ]:
dat_cords[V.FNAC] = dat_cords.apply(
    lambda x: f'{x[V.SESSIONID]}_s1_p{x[V.PANOID]}_r{x[V.ACROIID]}_a{x[V.ACID]}_ac_spheresegalign',axis=1)

In [ ]:
dat_cords.iloc[100]

In [ ]:
dat_cords.to_csv(Cout.fn_cords, index=False)

Get crop metadata

In [ ]:
fns = list(pathlib.Path(Cin.fol_crop).glob(Cparams.fn_crop_glob))
dat_crops = lib.get_crop_meta(fns, Cparams.re_crop)

In [ ]:
dat_crops.to_csv(Cout.fn_cropmeta, index=False)

Save slidescan scene images

In [ ]:
for fn in dat_scene_cord[V.SLIDESCAN].unique():
    lib.save_scene_images(pathlib.Path(Cin.fol_slide) / f'{fn}{V.SUFFIX_CZI}', Cout.fol_out_imgs)

In [ ]:
import sys
!conda env export -p {sys.prefix} 